In [1]:
import pandas as pd
import altair as alt
import numpy as np
from nltk.stem import PorterStemmer
ps = PorterStemmer()

# Ranking comparison - surverys and dueuling-bandints

## Read in data

In [2]:
df_free = pd.read_csv('../clean_data/free_choice_emotion_uw_students.csv')
df_free_labels = pd.read_csv('../data/emotion_labels_free_choice.csv')

df_forced = pd.read_csv('../clean_data/forced_choice_emotion_uw_students.csv')
df_forced_labels = pd.read_csv('../data/emotion_labels.csv')

In [3]:
df_emo_answers_free = df_free.loc[:, 'Q2.1_1':'Q195.1_4'] # get cols with words only
df_emo_answers_forced = df_forced.loc[:, 'Q2.1':'Q195.1'] 

In [4]:
def emotion_df_formated(df_emo_answers, emotion_label, df_labels):
    df_emo_cat = df_emo_answers.copy() 
    df_emo_cat_t = df_emo_cat.T # transpose
    df_emo_cat_t['photo_id'] = df_emo_cat_t.index # get index as col
    df_emo_cat_t = df_emo_cat_t.reset_index(drop=True) # clean index
    df_emo_cat_t_labels = pd.concat([df_emo_cat_t, df_labels], axis=1) # add metadata cols
    df_label =  df_emo_cat_t_labels[df_emo_cat_t_labels['label'] == emotion_label]
    
    return df_label

In [5]:
def formating_words(df, len_words=1, len_letters=2):
    df_stack = df.stack().reset_index(drop=True) # stack as series
    df_stack = df_stack.to_frame(name='emotion') # as DF
    df_stack['emotion'] = df_stack['emotion'].str.strip() # remove blank spaces
    df_stack['emotion'] = df_stack['emotion'].str.lower() # as lower case
    df_stack['emotion'] = df_stack['emotion'].replace({'na':'none'}) 
    df_stack['len_words'] = df_stack['emotion'].str.split().apply(len) # cnt number of words
    df_stack['len_letters'] = df_stack['emotion'].apply(len) # cont number of letters
    # get df with single words of 3 or more letters
    df_stack_single_word = df_stack[(df_stack['len_words'] == len_words) & (df_stack['len_letters'] > len_letters)] 
    return df_stack_single_word

In [6]:
def simple_per_bar_concat(
    df, title='Title', X='percent:Q', Y='emotion:N', \
    width=450, height=250, sort='-x', \
    text_size = 12, label_size = 11, title_size=12, \
    emotion='Some', color1='#0570b0', color2='orange'):
    
    bars = alt.Chart(df, title=title).mark_bar().encode(
        alt.X(X, axis=alt.Axis(format='.0%')),
        y=alt.Y(Y, sort=sort), 
        color=alt.condition(
            alt.datum.emotion == emotion,
            alt.value(color2),
            alt.value(color1)
        ))
    
    text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3,  # Nudges text to right so it doesn't appear on top of the bar
    fontSize=text_size
    ).encode(
        alt.Text(X, format='.1%')
    )
    
    chart = (bars + text)
    
    return chart

In [7]:
def simple_cnt_bar_concat(
    df, title='Title', X='percent:Q', Y='emotion:N', \
    width=450, height=250, sort='-x', \
    text_size = 12, label_size = 11, title_size=12, \
    emotion='Some', color1='#0570b0', color2='orange'):
    
    bars = alt.Chart(df, title=title).mark_bar().encode(
        alt.X(X, axis=alt.Axis(format='.0%')),
        y=alt.Y(Y, sort=sort), 
        color=alt.condition(
            alt.datum.emotion == emotion,
            alt.value(color2),
            alt.value(color1)
        ))
    
    text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3,  # Nudges text to right so it doesn't appear on top of the bar
    fontSize=text_size
    ).encode(
        alt.Text(X)
    )
    
    chart = (bars + text)
    
    return chart

In [10]:
def count_freq_labels(df, X="all", col="emotion"):
    if X == "all":
        df_counts = df.stack().reset_index(drop=True).value_counts() # stack as series
        df_counts = df_counts.to_frame('counts') # get value_counts as df
        df_counts['emotion'] = df_counts.index # get index as col
    elif X == "stacked":
        df_counts = df[col].value_counts() # count word frequency
        df_counts = df_counts.to_frame('counts') # get value_counts as df
        df_counts['emotion'] = df_counts.index # get index as col    
    else:
        df_counts = df[X].reset_index(drop=True).value_counts() # stack as series
        df_counts = df_counts.to_frame('counts') # get value_counts as df
        df_counts[X] = df_counts.index # get index as col

    df_counts = df_counts.reset_index(drop=True) # clean index
    df_counts['percent'] = df_counts['counts'] / df_counts['counts'].sum() # compute percentage
    return df_counts

In [11]:
def photo_chart(df_emo_answers, emotion,  i, df_labels, emotion_st = None, n_mentions=2, photo = None):
    df = emotion_df_formated(df_emo_answers, emotion, df_labels) # add emotion label
    photo_id_list = df['photo_id'].tolist() # photo id to list
    splited_photo_id = np.array_split(photo_id_list, 24) # split photo id by photo
    df = df[df['photo_id'].isin(splited_photo_id[i])] # select photo rows   
    df_url = df[['url']].head(1)  # get url for chart
    
    face = alt.Chart(df_url).mark_image(width=110, height=110, align='right', xOffset=0, yOffset=230).encode(url='url')
    
    df_emotion_ans = df.drop(['photo_id', 'ethnicity', 'sex', 'age', 'label', 'url', 'photoId'], axis=1) # clean cols
    df_stack_emotion = formating_words(df_emotion_ans) # clean up words
    df_stack_emotion['emotion_ps_steamed'] = df_stack_emotion['emotion'].apply(lambda x: ps.stem(x)) # steam
    df_stack_emotion_count = count_freq_labels(df_stack_emotion, X="stacked", col='emotion_ps_steamed') # group and count
    source = df_stack_emotion_count[df_stack_emotion_count['counts'] >= n_mentions]
    
    chart = simple_per_bar_concat(source,\
                           title=f"Categories with {n_mentions} or more mentions | n= {source['counts'].sum().astype(str)}", \
                           X='percent:Q', Y='emotion:N', emotion=emotion_st)

    if photo == True:
        return chart + face
    else:
        return chart

In [12]:
def next_chart_wrapper(file_path, emotion, head):
    df = pd.read_csv(file_path)
    
    df = df.round(decimals=2)

    df.rename(columns={
        'Target': 'emotion',
        'Score': 'borda score'}, inplace=True)

    title=f"Dueling bandits ranking"

    chart = simple_cnt_bar_concat(df.head(head), title= title, \
                                  X='borda score:Q', Y='emotion:N', emotion=emotion)
    return chart

## Anger - Female of color

In [13]:
anger_bfa_forced = photo_chart(df_emo_answers_forced, 'anger',  0, df_forced_labels, emotion_st='anger', n_mentions=2)
anger_bfa_free = photo_chart(df_emo_answers_free, 'anger',  0, df_free_labels, emotion_st='angri', n_mentions=2)

In [14]:
path = '../../emo-ranking-next/anger_bipoc_female/borda_lilucb_ranking.csv'

anger_bfa_next = next_chart_wrapper(path, emotion='angry', head=15)

In [15]:
(anger_bfa_forced.properties(width=200) | anger_bfa_free.properties(width=200) | anger_bfa_next.properties(width=200))

alt.HConcatChart(...)

## Anger - Male of color

In [16]:
anger_bma_forced = photo_chart(df_emo_answers_forced, 'anger',  2, df_forced_labels, emotion_st='anger', n_mentions=2)
anger_bma_free = photo_chart(df_emo_answers_free, 'anger',  2, df_free_labels, emotion_st='angri', n_mentions=2)

In [17]:
path = '../../emo-ranking-next/anger_bipoc_male/borda_lilucb_ranking.csv'

anger_bma_next = next_chart_wrapper(path, emotion='angry', head=15)

In [18]:
(anger_bma_forced.properties(width=200) | anger_bma_free.properties(width=200) | anger_bma_next.properties(width=200))

alt.HConcatChart(...)

## Anger - White Female 

In [19]:
anger_wfa_forced = photo_chart(df_emo_answers_forced, 'anger',  12, df_forced_labels, emotion_st='anger', n_mentions=2)
anger_wfa_free = photo_chart(df_emo_answers_free, 'anger',  12, df_free_labels, emotion_st='angri', n_mentions=2)

In [20]:
path = '../../emo-ranking-next/anger_white_female/borda_lilucb_ranking.csv'

anger_wfa_next = next_chart_wrapper(path, emotion='angry', head=15)

In [21]:
(anger_wfa_forced.properties(width=200) | anger_wfa_free.properties(width=200) | anger_wfa_next.properties(width=200))

alt.HConcatChart(...)

## Anger - White male

In [22]:
anger_wma_forced = photo_chart(df_emo_answers_forced, 'anger',  6, df_forced_labels, emotion_st='anger', n_mentions=2)
anger_wma_free = photo_chart(df_emo_answers_free, 'anger',  6, df_free_labels, emotion_st='angri', n_mentions=2)

In [23]:
path = '../../emo-ranking-next/anger_white_male/borda_lilucb_ranking.csv'

anger_wma_next = next_chart_wrapper(path, emotion='angry', head=28)

In [24]:
(anger_wma_forced.properties(width=200) | anger_wma_free.properties(width=200) | anger_wma_next.properties(width=200))

alt.HConcatChart(...)

## Disgust - Female of color

## Disgust - Male of color

## Disgust - White female

## Disgust - Whie male